In [6]:
from pyspark import SparkContext
from pyspark.sql import SparkSession,SQLContext

from pyspark.sql.types import StringType,StructField,IntegerType,StructType,FloatType,Row



In [ ]:
spark = SparkContext(master="local",appName="DataFrames")
sqlContext = SQLContext(spark)

In [14]:
juegoSchema = StructType([
    StructField("juego_id",IntegerType(),False),
    StructField("anio",StringType(),False),
    StructField("temporada",StringType(),False),
    StructField("ciudad",StringType(),False)
])
juegoDF = sqlContext.read.schema(juegoSchema)\
    .option("header","true").csv('./files/juegos.csv')

In [19]:
juegoDF.show(4)

22/10/26 23:12:24 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 5, schema size: 4
CSV file: file:///home/osmait/Documentos/Work-space/python/etl/etl_animes/files/juegos.csv
+--------+-----------+---------+------+
|juego_id|       anio|temporada|ciudad|
+--------+-----------+---------+------+
|       1|1896 Verano|     1896|Verano|
|       2|1900 Verano|     1900|Verano|
|       3|1904 Verano|     1904|Verano|
|       4|1906 Verano|     1906|Verano|
+--------+-----------+---------+------+
only showing top 4 rows



In [26]:

sqlContext.read.option("delimiter",",").option("header",True).csv('files/evento.csv').show(5)

+---------+--------------------+----------+
|evento_id|              evento|deporte_id|
+---------+--------------------+----------+
|        1|Basketball Men's ...|         1|
|        2|Judo Men's Extra-...|         2|
|        3|Football Men's Fo...|         3|
|        4|Tug-Of-War Men's ...|         4|
|        5|Speed Skating Wom...|         5|
+---------+--------------------+----------+
only showing top 5 rows



In [28]:
def eliminarEncabezado(indice, iterador):
    return iter(list(iterador)[1:])

In [31]:
deportistasOlimpicosRDD = spark.textFile("./files/deportista.csv").map(lambda l : l.split(','))
deportistasOlimpicosRDD2 = spark.textFile('./files/deportista2.csv').map(lambda l: l.split('/'))

deportistasOlimpicosRDD = deportistasOlimpicosRDD.union(deportistasOlimpicosRDD2)

In [33]:
deportistasOlimpicosRDD = deportistasOlimpicosRDD.mapPartitionsWithIndex(eliminarEncabezado)

In [36]:
deportistasOlimpicosRDD.take(2)

[['2', 'A Lamusi', '1', '23', '170', '60', '199'],
 ['3', 'Gunnar Nielsen Aaby', '1', '24', '0', '0', '273']]

In [52]:
deportistasOlimpicosRDD = deportistasOlimpicosRDD.map(lambda l: (
    int(l[0]),
    l[1],
    int(l[2]),
    int(l[3]),
    int(l[4]),
    float(l[5]),
    int(l[6])
))


In [57]:
schemaDeportistas = StructType([
    StructField("deportista_id",IntegerType(),False),
    StructField("nombre",StringType(),False),
    StructField("genero",IntegerType(),False),
    StructField("edad",IntegerType(),False),
    StructField("altura",IntegerType(),False),
    StructField("peso",FloatType(),False),
    StructField("equipo_id",IntegerType(),False)
])

In [60]:
deportistasOlimpicosDF = sqlContext.createDataFrame(deportistasOlimpicosRDD,schemaDeportistas)

In [62]:
deportistasOlimpicosDF.show(30)

+-------------+--------------------+------+----+------+-----+---------+
|deportista_id|              nombre|genero|edad|altura| peso|equipo_id|
+-------------+--------------------+------+----+------+-----+---------+
|            2|            A Lamusi|     1|  23|   170| 60.0|      199|
|            3| Gunnar Nielsen Aaby|     1|  24|     0|  0.0|      273|
|            4|Edgar Lindenau Aabye|     1|  34|     0|  0.0|      278|
|            5|Christine Jacoba ...|     2|  21|   185| 82.0|      705|
|            6|     Per Knut Aaland|     1|  31|   188| 75.0|     1096|
|            7|        John Aalberg|     1|  31|   183| 72.0|     1096|
|            8|Cornelia Cor Aalt...|     2|  18|   168|  0.0|      705|
|            9|    Antti Sami Aalto|     1|  26|   186| 96.0|      350|
|           10|Einar Ferdinand E...|     1|  26|     0|  0.0|      350|
|           11|  Jorma Ilmari Aalto|     1|  22|   182| 76.5|      350|
|           12|   Jyri Tapani Aalto|     1|  31|   172| 70.0|   